## Preparing Data

In [ ]:
from sqlalchemy.engine import create_engine
from sqlalchemy import inspect
from sqlalchemy import text
import csv

from keys_list import aws_access_key_id, aws_secret_access_key, unsubscribe_secret
from email.utils import COMMASPACE
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import boto.ses
import jinja2
import hashlib

from config import *

def micromasters_courses_add_last_run():
    for course in micromasters_courses:
        query = "SELECT course_instance FROM course_instancedetails WHERE course='%s' ORDER BY year DESC, term DESC;" % (course)
        query_result = sqlite_connection.execute(query).fetchone()
        if query_result is not None:
            micromasters_courses[course]['last_run'] = query_result[0]
    
def find_auditpassed(course):
    trigger_run = pointed_course_instances[course]['current_run']['instance_code']
    trigger_course_title = pointed_course_instances[course]['current_run']['title']
    trigger_course_link = pointed_course_instances[course]['current_run']['link']
    
    query = "SELECT c.learner_id, c.course_instance, c.audit_passed, l.name, l.email FROM course_instancedetails c, learners l WHERE c.learner_id = l.student_id AND c.course_instance = '%s' AND c.audit_passed = 'Y'" % (trigger_run)
    #print('find_auditpassed', query)
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        result = {}
        result['email'] = record['email']
        result['student_name'] = record['name']
        result['student_id'] = record['learner_id']
        result['trigger_course_verified'] = 0
        result['trigger_course_title'] = trigger_course_title
        result['trigger_course_link'] = trigger_course_link
        filtered_students.append(result)
    
def find_certificate_eligible(course):
    trigger_run = pointed_course_instances[course]['current_run']['instance_code']
    trigger_course_title = pointed_course_instances[course]['current_run']['title']
    trigger_course_link = pointed_course_instances[course]['current_run']['link']

    query = "SELECT c.learner_id, c.course_instance, c.audit_passed, l.name, l.email FROM course_instancedetails c, learners l WHERE c.learner_id = l.student_id AND c.course_instance = '%s' AND c.certificate_eligible = 'Y'" % (trigger_run)
    #print('find_certificate_eligible', query)
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        result = {}
        result['email'] = record['email']
        result['student_name'] = record['name']
        result['student_id'] = record['learner_id']
        result['trigger_course_verified'] = 1
        result['trigger_course_title'] = trigger_course_title
        result['trigger_course_link'] = trigger_course_link
        filtered_students.append(result)


def find_auditpassed_next_course(course, record):
    #print('find_next_course a', course, record)
    not_certificated = []
    
    if not (record[5] == 'verified' and record[6] == 'Y'):
        not_certificated.append(micromasters_course_series[0])
    if not (record[7] == 'verified' and record[8] == 'Y'):
        not_certificated.append(micromasters_course_series[1])
    if not (record[9] == 'verified' and record[10] == 'Y'):
        not_certificated.append(micromasters_course_series[2])
    if not (record[11] == 'verified' and record[12] == 'Y'):
        not_certificated.append(micromasters_course_series[3])
    #print(not_certificated)
    if course in not_certificated:
        not_certificated.remove(course)
    return not_certificated[0]

def find_certificate_eligible_next_course(record):
    #print('find_next_course c', course, record)
    not_certificated = []
    
    if not (record[5] == 'verified' and record[6] == 'Y'):
        not_certificated.append(micromasters_course_series[0])
    if not (record[7] == 'verified' and record[8] == 'Y'):
        not_certificated.append(micromasters_course_series[1])
    if not (record[9] == 'verified' and record[10] == 'Y'):
        not_certificated.append(micromasters_course_series[2])
    if not (record[11] == 'verified' and record[12] == 'Y'):
        not_certificated.append(micromasters_course_series[3])
    #print(not_certificated)
    return not_certificated[0]

def render(template_file, context_dict):
    template = templateEnv.get_template(template_file)
    output = template.render(item=context_dict)
    #print(output)
    return output

def get_md5_unsubsecret(email_opt_out):
    m = hashlib.md5()
    m.update((unsubscribe_secret + email_opt_out).encode('utf-8'))
    return m.hexdigest()

def make_unsubscribe_link(email_opt_out):
    email_opt_out = email_opt_out.lower()

    validation_hash = get_md5_unsubsecret(email_opt_out)

    link = 'https://tools.ceit.uq.edu.au/uqxmailouts/unsubscribe/index.php?email=' + email_opt_out + '&validation_hash=' + validation_hash
    #print('link', link)
    return link

class SESMessage(object):

    def __init__(self, source, to_addresses, subject, body_html, body_text, **kw):

        self.ses = boto.ses.connect_to_region('us-west-2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

        self._source = source
        self._to_addresses = to_addresses
        self._cc_addresses = None
        self._bcc_addresses = None

        self.subject = subject
        self.text = body_text
        self.html = body_html
        self.attachments = []

    def send(self):
        #print('sending')        
        if not self.ses:
            raise Exception('No connection found')

        if (self.text and not self.html and not self.attachments) or\
           (self.html and not self.text and not self.attachments):
            return self.ses.send_email(self._source, self.subject,
                self.text or self.html,
                self._to_addresses, self._cc_addresses,
                self._bcc_addresses,
                format='text' if self.text else 'html')
        else:
            if not self.attachments:
                message = MIMEMultipart('alternative')
                #message = MIMEMultipart()
                message['subject'] = self.subject
                message['from'] = self._source
                if isinstance(self._to_addresses, (list, tuple)):
                    message['To'] = COMMASPACE.join(self._to_addresses)
                else:
                    message['To'] = self._to_addresses

                message.attach(MIMEText(self.text, 'plain'))
                message.attach(MIMEText(self.html, 'html'))
                #print(message)
            else:
                raise NotImplementedError
        return self.ses.send_raw_email(message.as_string(), source=self._source, destinations=self._to_addresses)

    
mmCourse = 'engy'
table_name = mmCourse + '_coursecompletion'  
micromasters_courses = mmCoursesMeta[mmCourse]
micromasters_course_series = mmCoursesSeries[mmCourse]
sqlite_engine = create_engine('sqlite:////Users/saisun/ProjectCode/uqx_tools/micromastersreports/databases/' + mmCourse + '_verified.db')
sqlite_connection = sqlite_engine.connect()
email_list = []

for course in micromasters_course_series:
    print('Course: ', course)
    filtered_students = []

    find_auditpassed(course)
    find_certificate_eligible(course)
    print('filtered_students length ', len(filtered_students))

    for item in filtered_students:
        query = "SELECT * FROM %s WHERE email = '%s'" % (table_name, item['email'])
        #print(query)
        query_result = sqlite_connection.execute(query)
        for record in query_result:
            item['no_certificate_eligible'] = record['no_certificate_eligible']
            item['next_course_title'] = ''
            item['next_course_link'] = ''

            # if the student has achieved all four course certified or more, we are not going to email him
            if item['no_certificate_eligible'] >= 4:
                print('removed: ', item['student_id'])
                continue

            if (item['trigger_course_verified'] == 0) and (item['no_certificate_eligible'] < 3):
                next_course = find_auditpassed_next_course(course, record)
                item['next_course_title'] = pointed_course_instances[next_course]['coming_run']['title']
                item['next_course_link'] = pointed_course_instances[next_course]['coming_run']['link']
            if (item['trigger_course_verified'] == 1) and (item['no_certificate_eligible'] < 4):
                next_course = find_certificate_eligible_next_course(record)
                item['next_course_title'] = pointed_course_instances[next_course]['coming_run']['title']
                item['next_course_link'] = pointed_course_instances[next_course]['coming_run']['link']
            email_list.append(item)
    #print('email_list', email_list)
    #print('email_list length ', len(email_list))
            
print(email_list)




## Send email out

In [ ]:

templateLoader = jinja2.FileSystemLoader(searchpath="./dynamic_email_templates")
templateEnv = jinja2.Environment(loader=templateLoader)

text_file = template_files[mmCourse] + '.txt'
html_file = template_files[mmCourse] + '.html'

#print(next_course)
test_user = [{'email': 'uqssun@uq.edu.au', 'student_name': 'Sai Sun', 'student_id': 10553000, 'trigger_course_verified': 0, 'trigger_course_title': pointed_course_instances[course]['current_run']['title'], 'trigger_course_link': pointed_course_instances[course]['current_run']['link'], 'no_certificate_eligible': 2, 'next_course_title': pointed_course_instances[next_course]['coming_run']['title'], 'next_course_link': pointed_course_instances[next_course]['coming_run']['link']}]
email_list = email_list + test_user
print(email_list)

#for item in email_list:         
for item in test_user:
    item['unsubscribe_link'] = make_unsubscribe_link(item['email'])
    #print(item)
    body_text = render(text_file, item)
    body_html = render(html_file, item)
    sesMessage = SESMessage(from_address, item['email'], subject, body_html, body_text)
    sesMessage.send()


print('The End.')
